# Main Configs

In [2]:
small_df = False
batch_size = 1
width, height = 300, 300

# Setup and Imports

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir /usr/lib/python3.7/metrics
!cp -R /content/drive/MyDrive/BA/Notebooks/2_Experiment/review_object_detection_metrics-main/src /usr/lib/python3.7/metrics/src

In [6]:
!pip install PyQt5
!pip install -qU torch_snippets

     |████████████████████████████████| 8.3 MB 9.9 MB/s 
     |████████████████████████████████| 59.9 MB 222 kB/s 
     |████████████████████████████████| 338 kB 51.9 MB/s 
     |████████████████████████████████| 48 kB 3.0 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 10.9 MB 58.1 MB/s 
     |████████████████████████████████| 948 kB 47.1 MB/s 
     |████████████████████████████████| 229 kB 53.0 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
     |████████████████████████████████| 60 kB 4.5 MB/s 
     |████████████████████████████████| 51 kB 6.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


In [7]:
!git clone https://github.com/sizhky/ssd-utils/
%cd ssd-utils

Cloning into 'ssd-utils'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.
/content/ssd-utils


In [8]:
import copy
import glob
import torch
import time
import statistics
import cv2
import pandas as pd
import IPython

from os.path import join
from torch_snippets import *
from torchvision.ops import nms
import torchvision.ops.boxes as bops
from PIL import Image
from metrics.src.evaluators import coco_evaluator, pascal_voc_evaluator
from metrics.src.bounding_box import BoundingBox
from metrics.src.utils.enumerators import BBFormat, BBType, CoordinatesType, MethodAveragePrecision
from skimage import data
from skimage.color import rgb2hsv, rgb2luv, rgb2lab
from skimage.util import random_noise

In [9]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    string = f'\r{prefix} |{bar}| {percent}% {suffix}'
    out.update(IPython.display.Pretty(string))
    # Print New Line on Complete
    if iteration == total: 
        print()

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
IMAGE_ROOT = '/content/drive/MyDrive/BA/dataset/bus-trucks/images'
OUTPUT_REPORTS = '/content/drive/MyDrive/BA/Notebooks/2_Experiment/Output_Reports'
OUTPUT_MODELS = '/content/drive/MyDrive/BA/Notebooks/2_Experiment/Output_Models'
output_training_report = 'ssd_vgg16.csv'
output_testing_report = 'testing_ssd_vgg16.xlsx'
output_model_name = 'ssd_vgg16.pt'

In [12]:
df_train = pd.read_csv('/content/drive/MyDrive/BA/dataset/Experimente/df_80_20_train_tf.csv')
df_test = pd.read_csv('/content/drive/MyDrive/BA/dataset/Experimente/df_80_20_test_tf.csv')
df_test.head()

,filename,class,xmin,xmax,ymin,ymax
0,00013f14dd4e168f.jpg,Bus,0.287500,0.999375,0.194184,0.999062
1,0002914fa805e227.jpg,Truck,0.061250,0.966875,0.125399,0.974495
2,0005f203463a13a8.jpg,Truck,0.000000,0.700000,0.187778,0.998889
3,00066517f9d814f9.jpg,Truck,0.000000,0.588867,0.069892,0.998208
4,000812dcf304a8e7.jpg,Bus,0.059375,0.848750,0.029936,0.958660


In [13]:
label2target = {l:t+1 for t,l in enumerate(df_train['class'].unique())}
label2target['background'] = 0
target2label = {t:l for l,t in label2target.items()}
label2target = {v: k for k, v in target2label.items()}
background_class = label2target['background']
num_classes = len(label2target)

print(target2label)

{1: 'Bus', 2: 'Truck', 0: 'background'}


In [ ]:
df_report = pd.read_csv(join(OUTPUT_REPORTS, output_training_report))
df_report.head()

,epoch,time,loss
0,0,0.557357,17.273149
1,0,0.271990,15.240695
2,0,0.279001,13.537352
3,0,0.313096,11.247987
4,0,0.267380,9.465345


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
from torchvision import transforms

def preprocess_image(img):
  img = torch.tensor(img).permute(2,0,1)
  return img.to(device).float()

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
denormalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

In [16]:
class OpenDataset(torch.utils.data.Dataset):
  w, h = width, height
  def __init__(self, df, image_dir=IMAGE_ROOT):
    self.image_dir = image_dir
    self.files = glob.glob(self.image_dir+'/*')
    self.df = df
    self.image_infos = df['filename'].unique()

  def __getitem__(self, ix):

    #filename	class	xmin	xmax	ymin	ymax

    # # load images and masks
    image_id = self.image_infos[ix]
    img_path = find(image_id, self.files)
    img = Image.open(img_path).convert('RGB')
    img = np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.

    data = self.df[self.df['filename'] == image_id]
    labels = data['class'].values.tolist()
    data = data[['xmin','ymin','xmax','ymax']].values
    data[:,[0,2]] *= self.w
    data[:,[1,3]] *= self.h
    boxes = data.astype(np.uint32).tolist() # convert to absolute coordinates
    return img, boxes, labels

  def collate_fn(self, batch):
    images, boxes, labels = [], [], []
    for item in batch:
      img, image_boxes, image_labels = item
      img = preprocess_image(img)[None]
      images.append(img)
      boxes.append(torch.tensor(image_boxes).float().to(device)/300.)
      labels.append(torch.tensor([label2target[c] for c in image_labels]).long().to(device))
    images = torch.cat(images).to(device)
    return images, boxes, labels

  def __len__(self):
    return len(self.image_infos)

In [ ]:
if small_df:
  df_test = df_test[:50]

In [23]:
test_ds = OpenDataset(df_test)
test_loader = DataLoader(test_ds, batch_size=batch_size, collate_fn=test_ds.collate_fn, drop_last=True)

In [18]:
from model import SSD300, MultiBoxLoss
from detect import *

# Load
model = SSD300(num_classes, device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy, device=device)
model.load_state_dict(torch.load(join(OUTPUT_MODELS, output_model_name), map_location=device))
# model.to(device);

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]


Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


<All keys matched successfully>

In [19]:
from torchvision.ops import nms
def decode_output(bbs, scores, labels):
  ixs = nms((torch.tensor(bbs)).type(torch.float64), torch.tensor(scores).type(torch.float64), 0.05)
  # bbs, scores, labels = [tensor[ixs] for tensor in [bbs, scores, labels]]
  bbs = [bb for i, bb in enumerate(bbs) if i in ixs]
  scores = [score for i, score in enumerate(scores) if i in ixs]
  labels = [label for i, label in enumerate(labels) if i in ixs]
  return bbs, scores, labels

# Size Analysis

In [ ]:
bounding_boxes_detected = []
bounding_boxes_gt = []
n_items = len(test_loader)
out = display(IPython.display.Pretty('Starting'), display_id=True)
model.eval()

for ix, (images, bbs, cls) in enumerate(test_loader):
  cls = cls[0]
  for i, box in enumerate(bbs[0]):
    cls_box = target2label[int(cls[i])]
    box = (box * width).type(torch.int64).tolist()
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls_box,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    bounding_boxes_gt.append(bb)

  image = Image.fromarray(np.uint8(images[0].cpu().permute(1, 2, 0).numpy() * 255))
  bbs, labels, scores = detect(image, model, min_score=0.0, max_overlap=0.05,top_k=200, device=device)
  bbs, scores, labels = decode_output(bbs, scores, labels.tolist())

  #Detected
  for i, box in enumerate(bbs):
    cls_box = target2label[int(labels[i])]
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls_box,
      coordinates       = box,
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.DETECTED,
      confidence        = scores[i],
      format            = BBFormat.XYX2Y2
    )
    bounding_boxes_detected.append(bb)

  printProgressBar(ix + 1, n_items, prefix = 'Progress:', suffix = 'Complete', length = 50)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
coco_evaluator.get_coco_summary(
    groundtruth_bbs = bounding_boxes_gt,
    detected_bbs = bounding_boxes_detected,
    small_size = 146,
    medium_size = 228
)

{'AP': 0.3241587786954479,
 'AP50': 0.5751419079669917,
 'AP75': 0.3317989507108984,
 'APlarge': 0.607574792916418,
 'APmedium': 0.49865651600497796,
 'APsmall': 0.1689894609957411,
 'AR1': 0.3546600851614429,
 'AR10': 0.3952168226508288,
 'AR100': 0.3956214017113159,
 'ARlarge': 0.6758517302407899,
 'ARmedium': 0.5853984414278532,
 'ARsmall': 0.23067560781146454}

# Full COCO

In [ ]:
df = coco_evaluator.get_coco_metrics(
        bounding_boxes_gt,
        bounding_boxes_detected,
        iou_threshold=0.5,
        area_range=(0, np.inf),
        max_dets=100,
)
df = pd.DataFrame.from_dict(df)
df.head()

,Bus,Truck
class,Bus,Truck
precision,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.3333333333333333, 0.5, 0.6, 0.666..."
recall,"[0.00043233895373973193, 0.0008646779074794639...","[0.0, 0.0, 0.0003996802557953637, 0.0007993605..."
AP,0.616386,0.533898
interpolated precision,"[1.0, 1.0, 1.0, 1.0, 0.9928571428571429, 0.992...","[0.9836065573770492, 0.9836065573770492, 0.983..."


# PASCAL VOL

In [ ]:
df = pascal_voc_evaluator.get_pascalvoc_metrics(
    bounding_boxes_gt,
    bounding_boxes_detected,
    iou_threshold= 0.1,
    method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION
)

df = pd.DataFrame.from_dict(df)
df.head()

,per_class,mAP
Bus,"{'precision': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1...",0.684875
Truck,"{'precision': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1...",0.684875


# Color Analysis

In [20]:
def image_light(img, thrshld = 127):
  if type(img) is torch.Tensor:
    img = img.cpu().permute(1,2,0).numpy()
  rgb = img * 255
  rgb = np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
  
  is_light = np.mean(rgb) > thrshld

  # Light: 0
  # Dark: 1

  # return 0 if is_light else 1
  return {'value': np.mean(rgb),
          'brightness': 'light' if is_light else 'dark'
          }

def avg_img_color(img):
  if type(img) is torch.Tensor:
    img = img.cpu().permute(1,2,0).numpy()
  rgb_avg = img.mean(axis=(0, 1)) 
  hsv_avg = rgb2hsv(img).mean(axis=(0, 1))
  luv_avg = rgb2luv(img).mean(axis=(0, 1))
  lab_avg = rgb2lab(img).mean(axis=(0, 1))

  return {
    'rgb_avg_r': float(rgb_avg[0]),
    'rgb_avg_g': float(rgb_avg[1]),
    'rgb_avg_b': float(rgb_avg[2]),
    'hsv_avg_h': float(hsv_avg[0]),
    'hsv_avg_s': float(hsv_avg[1]),
    'hsv_avg_v': float(hsv_avg[2])
  }

def get_sharpness(img):
  if type(img) is torch.Tensor:
    img = img.cpu().permute(1,2,0).numpy()
  i = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])

  gy, gx = np.gradient(i)
  gnorm = np.sqrt(gx**2 + gy**2)
  sharpness = np.average(gnorm)
  return sharpness

In [21]:
def bb_intersection_over_union(boxA, boxB):

  # xA = max(boxA[0], boxB[0])
  # yA = max(boxA[1], boxB[1])
  # xB = min(boxA[2], boxB[2])
  # yB = min(boxA[3], boxB[3])

  # interArea = (xB - xA) * (yB - yA)

  # boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
  # boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

  # iou = interArea / float(boxAArea + boxBArea - interArea)
  iou = bops.box_iou(torch.tensor([boxA]), torch.tensor([boxB]))
  return iou.item()


def detected(preds, gt, iou_threshold ):
  det = False
  max_conf = 0
  for i, pred in enumerate(preds):
    if pred[0] == gt[0]:
      if pred[1] == gt[1]:
        iuo = bb_intersection_over_union(pred[-4:], gt[-4:])
        if iuo >= iou_threshold:
          det = True
          if pred[2] > max_conf:
            max_conf = pred[2]

  return det, max_conf

In [24]:
blur_filter = 1/9 *  torch.tensor([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
metadata = []
n_items = len(test_loader)
out = display(IPython.display.Pretty('Starting'), display_id=True)
model.eval()

for ix, (images, bbs, cls) in enumerate(test_loader):
  
  cls_gt = cls[0]
  img = images[0]
  img_ = Image.fromarray(np.uint8(img.cpu().permute(1, 2, 0).numpy() * 255))


  # plain image
  c_img = image_light(img)
  c_avg_img = avg_img_color(img)

  preds = []
  
  # image = Image.fromarray(np.uint8(img_.numpy() * 255))
  bbs_output, labels_output, scores_output = detect(img_, model, min_score=0.01, max_overlap=0.05,top_k=200, device=device)
  bbs_output, scores_output, labels_output = decode_output(bbs_output, scores_output, labels_output.tolist())


  for i_bb, bb in enumerate(bbs_output):
    X1, Y1, X2, Y2 = bb
    l = target2label[labels_output[i_bb]]
    preds.append([ix, l, scores_output[i_bb], X1, Y1, X2, Y2])

  for i, box in enumerate(bbs[0]):
    
    box_gt = (box * width).type(torch.int64).tolist()
    X1, Y1, X2, Y2 = box_gt
      
    cls_ = target2label[int(cls_gt[i])]

    # object frame
    obj_frame = img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box_gt),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls_, 1, X1, Y1, X2, Y2],
        0.6
    )

    metadata.append({'image_id': ix,
                    'image_version': 'standard',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls_
                   }
                  )
###########################################################################################

  # blurred image
  blurred_img = cv2.filter2D(img.cpu().permute(1, 2, 0).numpy(), -1, kernel = blur_filter.numpy())
  c_img = image_light(blurred_img)
  c_avg_img = avg_img_color(blurred_img)

  blurred_imgs = [torch.from_numpy(blurred_img).permute(2, 0, 1)]
  blurred_img = blurred_imgs[0]
  blurred_img_ = Image.fromarray(np.uint8(blurred_img.cpu().permute(1, 2, 0).numpy() * 255))
  
  preds = []
  
  bbs_output, labels_output, scores_output = detect(blurred_img_, model, min_score=0.01, max_overlap=0.05, top_k=200, device=device)
  bbs_output, scores_output, labels_output = decode_output(bbs_output, scores_output, labels_output.tolist())

  for i_bb, bb in enumerate(bbs_output):
    X1, Y1, X2, Y2 = bb
    l = target2label[labels_output[i_bb]]
    preds.append([ix, l, scores_output[i_bb], X1, Y1, X2, Y2])

  for i, box in enumerate(bbs[0]):
    
    box_gt = (box * width).type(torch.int64).tolist()
    X1, Y1, X2, Y2 = box_gt
      
    cls_ = target2label[int(cls_gt[i])]

    # object frame
    obj_frame = blurred_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box_gt),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls_, 1, X1, Y1, X2, Y2],
        0.6
    )

    metadata.append({'image_id': ix,
                    'image_version': 'blurred',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls_
                   }
                  )

###########################################################################################

  # noisy image

  noise_img = random_noise(img.cpu().permute(1, 2, 0).numpy(), mode='s&p',amount=0.05)
  c_img = image_light(noise_img)
  c_avg_img = avg_img_color(noise_img)

  noisy_imgs = [torch.from_numpy(noise_img).permute(2, 0, 1)]
  noisy_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in noisy_imgs]
  noisy_img = noisy_imgs[0]
  noisy_img_ = Image.fromarray(np.uint8(blurred_img.cpu().permute(1, 2, 0).numpy() * 255))
  
  preds = []
  
  bbs_output, labels_output, scores_output = detect(noisy_img_, model, min_score=0.01, max_overlap=0.05, top_k=200, device=device)
  bbs_output, scores_output, labels_output = decode_output(bbs_output, scores_output, labels_output.tolist())

  for i_bb, bb in enumerate(bbs_output):
    X1, Y1, X2, Y2 = bb
    l = target2label[labels_output[i_bb]]
    preds.append([ix, l, scores_output[i_bb], X1, Y1, X2, Y2])

  for i, box in enumerate(bbs[0]):
    
    box_gt = (box * width).type(torch.int64).tolist()
    X1, Y1, X2, Y2 = box_gt
      
    cls_ = target2label[int(cls_gt[i])]

    # object frame
    obj_frame = noisy_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box_gt),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls_, 1, X1, Y1, X2, Y2],
        0.6
    )

    metadata.append({'image_id': ix,
                    'image_version': 'noisy',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls_
                   }
                  )

###########################################################################################

  # bright image

  new_image = PIL.ImageEnhance.Brightness(img_).enhance(2)
  bright_image = np.array(new_image) / 255
  bright_imgs = [torch.from_numpy(bright_image).permute(2, 0, 1)]
  bright_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in bright_imgs]
  bright_img = bright_imgs[0]

  c_img = image_light(bright_image)
  c_avg_img = avg_img_color(bright_image)
  
  preds = []
  
  bbs_output, labels_output, scores_output = detect(new_image, model, min_score=0.01, max_overlap=0.05, top_k=200, device=device)
  bbs_output, scores_output, labels_output = decode_output(bbs_output, scores_output, labels_output.tolist())

  for i_bb, bb in enumerate(bbs_output):
    X1, Y1, X2, Y2 = bb
    l = target2label[labels_output[i_bb]]
    preds.append([ix, l, scores_output[i_bb], X1, Y1, X2, Y2])

  for i, box in enumerate(bbs[0]):
    
    box_gt = (box * width).type(torch.int64).tolist()
    X1, Y1, X2, Y2 = box_gt
      
    cls_ = target2label[int(cls_gt[i])]

    # object frame
    obj_frame = bright_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box_gt),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls_, 1, X1, Y1, X2, Y2],
        0.6
    )

    metadata.append({'image_id': ix,
                    'image_version': 'bright',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls_
                   }
                  )



###########################################################################################

  # contrast image

  new_image = PIL.ImageEnhance.Contrast(img_).enhance(2)
  contrast_image = np.array(new_image) / 255
  contrast_imgs = [torch.from_numpy(contrast_image).permute(2, 0, 1)]
  contrast_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in contrast_imgs]
  contrast_img = contrast_imgs[0]

  c_img = image_light(contrast_image)
  c_avg_img = avg_img_color(contrast_image)
  
  preds = []
  
  bbs_output, labels_output, scores_output = detect(new_image, model, min_score=0.01, max_overlap=0.05, top_k=200, device=device)
  bbs_output, scores_output, labels_output = decode_output(bbs_output, scores_output, labels_output.tolist())

  for i_bb, bb in enumerate(bbs_output):
    X1, Y1, X2, Y2 = bb
    l = target2label[labels_output[i_bb]]
    preds.append([ix, l, scores_output[i_bb], X1, Y1, X2, Y2])

  for i, box in enumerate(bbs[0]):
    
    box_gt = (box * width).type(torch.int64).tolist()
    X1, Y1, X2, Y2 = box_gt
      
    cls_ = target2label[int(cls_gt[i])]

    # object frame
    obj_frame = contrast_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box_gt),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls_, 1, X1, Y1, X2, Y2],
        0.6
    )

    metadata.append({'image_id': ix,
                    'image_version': 'contrast',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls_
                   }
                  )


###########################################################################################

  # sharp image

  new_image = PIL.ImageEnhance.Sharpness(img_).enhance(3)
  sharp_image = np.array(new_image) / 255
  sharp_imgs = [torch.from_numpy(sharp_image).permute(2, 0, 1)]
  sharp_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in sharp_imgs]
  sharp_img = sharp_imgs[0]

  c_img = image_light(sharp_image)
  c_avg_img = avg_img_color(sharp_image)
  
  preds = []
  
  bbs_output, labels_output, scores_output = detect(new_image, model, min_score=0.01, max_overlap=0.05, top_k=200, device=device)
  bbs_output, scores_output, labels_output = decode_output(bbs_output, scores_output, labels_output.tolist())

  for i_bb, bb in enumerate(bbs_output):
    X1, Y1, X2, Y2 = bb
    l = target2label[labels_output[i_bb]]
    preds.append([ix, l, scores_output[i_bb], X1, Y1, X2, Y2])

  for i, box in enumerate(bbs[0]):
    
    box_gt = (box * width).type(torch.int64).tolist()
    X1, Y1, X2, Y2 = box_gt
      
    cls_ = target2label[int(cls_gt[i])]

    # object frame
    obj_frame = sharp_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box_gt),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls_, 1, X1, Y1, X2, Y2],
        0.6
    )

    metadata.append({'image_id': ix,
                    'image_version': 'sharp',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls_
                   }
                  )
    

###########################################################################################

  # dark image

  new_image = PIL.ImageEnhance.Brightness(img_).enhance(0.4)
  dark_image = np.array(new_image) / 255
  dark_imgs = [torch.from_numpy(dark_image).permute(2, 0, 1)]
  dark_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in dark_imgs]
  dark_img = dark_imgs[0]

  c_img = image_light(dark_image)
  c_avg_img = avg_img_color(dark_image)
  
  preds = []
  
  bbs_output, labels_output, scores_output = detect(new_image, model, min_score=0.01, max_overlap=0.05, top_k=200, device=device)
  bbs_output, scores_output, labels_output = decode_output(bbs_output, scores_output, labels_output.tolist())

  for i_bb, bb in enumerate(bbs_output):
    X1, Y1, X2, Y2 = bb
    l = target2label[labels_output[i_bb]]
    preds.append([ix, l, scores_output[i_bb], X1, Y1, X2, Y2])

  for i, box in enumerate(bbs[0]):
    
    box_gt = (box * width).type(torch.int64).tolist()
    X1, Y1, X2, Y2 = box_gt
      
    cls_ = target2label[int(cls_gt[i])]

    # object frame
    obj_frame = dark_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box_gt),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls_, 1, X1, Y1, X2, Y2],
        0.6
    )

    metadata.append({'image_id': ix,
                    'image_version': 'dark',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls_
                   }
                  )
  printProgressBar(ix + 1, n_items, prefix = 'Progress:', suffix = 'Complete', length = 50)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:173: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:251: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:333: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:414: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_

In [ ]:
# show(img)
# show(blurred_img)
# show(noise_img)
# show(bright_img)
# show(contrast_img)
# show(sharp_img)
# show(dark_img)

Output hidden; open in https://colab.research.google.com to view.

In [26]:
df = pd.DataFrame.from_dict(metadata)
df.to_excel(join(OUTPUT_REPORTS, output_testing_report))
df.tail(7)

,image_id,image_version,image_brightness_value,image_brightness_interpretation,image_rgb_avg_r,image_rgb_avg_g,image_rgb_avg_b,image_hsv_avg_h,image_hsv_avg_s,image_hsv_avg_v,object_brightness_value,object_brightness_interpretation,object_rgb_avg_r,object_rgb_avg_g,object_rgb_avg_b,object_hsv_avg_h,object_hsv_avg_s,object_hsv_avg_v,detected,confidence,object_size,object_class
33698,3044,bright,134.776908,light,0.552707,0.519490,0.512211,0.254606,0.208871,0.568104,219.471943,light,0.900724,0.844611,0.839133,0.215730,0.116299,0.918348,False,0.000000,10486.0,Bus
33699,3044,contrast,95.425101,dark,0.405319,0.362328,0.354209,0.183127,0.219872,0.419370,198.343621,light,0.790550,0.769863,0.786081,0.294754,0.135345,0.818334,False,0.000000,2240.0,Bus
33700,3044,contrast,95.425101,dark,0.405319,0.362328,0.354209,0.183127,0.219872,0.419370,175.065043,light,0.755075,0.657424,0.657277,0.318293,0.271466,0.785582,False,0.000000,10486.0,Bus
33701,3044,sharp,82.461668,dark,0.343913,0.316154,0.307026,0.338734,0.255500,0.356855,151.929901,light,0.607832,0.591539,0.586746,0.379261,0.131465,0.630860,False,0.000000,2240.0,Bus
33702,3044,sharp,82.461668,dark,0.343913,0.316154,0.307026,0.338734,0.255500,0.356855,137.296597,light,0.574613,0.523310,0.521784,0.416595,0.209666,0.603817,True,0.969661,10486.0,Bus
33703,3044,dark,32.575656,dark,0.136002,0.124840,0.121187,0.313693,0.260449,0.140653,60.404664,dark,0.241760,0.235164,0.233139,0.361929,0.113127,0.249445,False,0.000000,2240.0,Bus
33704,3044,dark,32.575656,dark,0.136002,0.124840,0.121187,0.313693,0.260449,0.140653,54.514213,dark,0.228268,0.207731,0.207138,0.402688,0.190001,0.238191,True,0.989545,10486.0,Bus


In [1]:
df.head(7)

NameError: ignored